In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import matrix
import model
import analysis as an
import plot

In [2]:
def analysis(init, patterns, end_time):
    """sparsity_analysis.
    Generate simulation data to analyze how K_ac
    sparsity affects model dynamics.

    :param init:  inital abudance vector
    :param pattern: matrix patterns to test
    :param end_time: time steps to simulate
    """
    rows, n = [], len(init)
    
    fixation_times = []
    for pattern in patterns:
        curr_mat = matrix.pattern_matrix(pattern, n)
        
        # simulate model with given params
        t, N, S, E, P = model.simulate(curr_mat, init, end_time)
        # N, S are variables by time points matrices
        
        
        # Calculate statistics
        row = {"pattern": pattern}
        
        row["initial_conditions"] = init
        
        # Compute time to fixation
        row['fixation_time'] = len(N[0])
        
        # Compute growth rate
        row["growth_rate"] = an.growth_rate(N.T)
        
        N = np.apply_along_axis(an.normalize, 1, N.T).T
        
        # Compute skewness 
        row["skewness"] = np.average(an.skewness(N.T))
        
        # Compute richness
        richnessVals = [an.richness(i) for i in N.T]
        row["richness"] = np.average(richnessVals)
        
        # Compute shannon index
        row["shannon_index"] = np.average(an.shannon_index(N.T))
        
        # Add Bray Curtis distance metric array for each K_ac simulation
        brayCurtisMtx = np.identity(n)
        for i in range(n):
            for j in range(i + 1, n):
                brayCurtisMtx[i, j] = an.bray_curtis(N[i], N[j])
        row["bray curtis"] = brayCurtisMtx
        
        # store for easy conversion to a dataframe
        rows.append(row)
        
    return pd.DataFrame(rows)

In [3]:
patterns = ['null', 'ident', 'naive', 'immune', 'barbell', 'cycle', 'complete', 'star']

In [4]:
# Create random inits
def make_random_init(num_strains):
    max_val = 0.5
    min_val = 0
    
    init = []
    for i in range(num_strains):
        init.append(np.random.uniform(min_val, max_val))
    
    return init

In [5]:
# Generate 100 random inits
inits = []
for i in range(100):
    inits.append(make_random_init(4))

In [ ]:
# Create dataframe containing statistics for all inits
big_df = pd.DataFrame()

for init in inits:
    res = analysis(init, patterns, 50)
    big_df = big_df.append(res)

big_df.reset_index(drop=True)

In [ ]:
# Create pattern dataframes
null_df = big_df.loc[big_df['pattern'] == 'null']
ident_df = big_df.loc[big_df['pattern'] == 'ident']
naive_df = big_df.loc[big_df['pattern'] == 'naive']
immune_df = big_df.loc[big_df['pattern'] == 'immune']
barbell_df = big_df.loc[big_df['pattern'] == 'barbell']
cycle_df = big_df.loc[big_df['pattern'] == 'cycle']
complete_df = big_df.loc[big_df['pattern'] == 'complete']
star_df = big_df.loc[big_df['pattern'] == 'star']

# Create list to hold all pattern dataframes
all_pattern_dfs = [null_df, ident_df, naive_df, immune_df, barbell_df, cycle_df, complete_df, star_df]

In [ ]:
# growth rate plots
# x axis = metric range, y axis = frequency

plt.figure(figsize=(16, 9))

i=1
for pattern_df in all_pattern_dfs:
    vals = pattern_df['growth_rate'].values
    plt.subplot(2, 4, i)
    plt.title(pattern_df['pattern'].values[0])
    sns.kdeplot(vals)
    i += 1
    
plt.suptitle('Growth Rate')
plt.show()

In [ ]:
# skewness plots
# x axis = metric range, y axis = frequency

plt.figure(figsize=(16, 9))

i=1
for pattern_df in all_pattern_dfs:
    vals = pattern_df['skewness'].values
    plt.subplot(2, 4, i)
    plt.title(pattern_df['pattern'].values[0])
    sns.kdeplot(vals)
    i += 1
    
plt.suptitle('Skewness')
plt.show()

In [ ]:
# shannon index plots
# x axis = metric range, y axis = frequency

plt.figure(figsize=(16, 9))

i=1
for pattern_df in all_pattern_dfs:
    vals = pattern_df['shannon_index'].values
    plt.subplot(2, 4, i)
    plt.title(pattern_df['pattern'].values[0])
    sns.kdeplot(vals)
    i += 1
    
plt.suptitle('Shannon Index')
plt.show()

In [ ]:
# fixation time plots
# x axis = metric range, y axis = frequency

plt.figure(figsize=(16, 9))

i=1
for pattern_df in all_pattern_dfs:
    vals = pattern_df['fixation_time'].values
    plt.subplot(2, 4, i)
    plt.title(pattern_df['pattern'].values[0])
    sns.kdeplot(vals)
    i += 1
    
plt.suptitle('Times to Fixation')
plt.show()

In [ ]:
# Melt big_df
melted_df = pd.melt(big_df, id_vars =['pattern'], value_vars =['growth_rate', 'skewness', 'shannon_index', 'fixation_time'])

In [ ]:
# plot all pattern matrices on one plot showing growth rate

plt.figure(figsize=(10, 6))
i=1
for pattern_df in all_pattern_dfs:
    vals = pattern_df['growth_rate'].values
    sns.kdeplot(vals, label=pattern_df['pattern'].values[0])
    plt.title('Growth Rate')
    i += 1
    
plt.show()


In [ ]:
# plot all pattern matrices on one plot showing skewness

plt.figure(figsize=(10, 6))
i=1
for pattern_df in all_pattern_dfs:
    vals = pattern_df['skewness'].values
    sns.kdeplot(vals, label=pattern_df['pattern'].values[0])
    plt.title('Skewness')
    i += 1
    
plt.show()

In [ ]:
# plot all pattern matrices on one plot showing shannon index

plt.figure(figsize=(10, 6))
i=1
for pattern_df in all_pattern_dfs:
    vals = pattern_df['shannon_index'].values
    sns.kdeplot(vals, label=pattern_df['pattern'].values[0])
    plt.title('Shannon Index')
    i += 1
    
plt.show()

In [ ]:
# plot all pattern matrices on one plot showing shannon index

plt.figure(figsize=(10, 6))
i=1
for pattern_df in all_pattern_dfs:
    vals = pattern_df['fixation_time'].values
    sns.kdeplot(vals, label=pattern_df['pattern'].values[0])
    plt.title('Fixation Time')
    i += 1

plt.legend()
plt.show()